In [1]:
import json
import os
import time
import random
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import collections

import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets
from torchvision import transforms
from torchvision import models
from torchsummary import summary

from model_pytorch import resnext50_32x4d
from utils import Bar,Logger, AverageMeter, accuracy, mkdir_p, savefig
from warmup_scheduler import GradualWarmupScheduler
from sklearn.metrics import accuracy_score, roc_auc_score
from sklearn.model_selection import train_test_split

from scipy.ndimage.filters import gaussian_filter
import cv2

from PIL import ImageFile, ImageOps
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [2]:
# GPU Device
gpu_id = 3
os.environ['CUDA_VISIBLE_DEVICES'] = str(gpu_id)
use_cuda = torch.cuda.is_available()
print("GPU device %d:" %(gpu_id), use_cuda)

GPU device 3: True


# Arguments

In [3]:
source_dir = '/media/data2/dataset/GAN_ImageData/StyleGAN2_256/'
target_dir = '/media/data2/dataset/GAN_ImageData/PGGAN_128/'

In [4]:
pretrained = './log/style2/128/32x4d/aug/checkpoint.pth.tar'
resume = ''

In [5]:
# Model
model_name = 'resnext32x4d' # b0-b7 scale

# Optimization
num_classes = 2
epochs = 1000
start_epoch = 0
train_batch = 125
test_batch = 125
lr = 0.1
schedule = [50, 250, 500, 750]
momentum = 0.1
gamma = 0.1 # LR is multiplied by gamma on schedule

# CheckPoint
checkpoint = './log/style2/128/32x4d/to_pggan/2000shot/self2' # dir
if not os.path.isdir(checkpoint):
    os.makedirs(checkpoint)
num_workers = 8

# Seed
manual_seed = 7
random.seed(manual_seed)
torch.cuda.manual_seed_all(manual_seed)

# Image
size = (128, 128)

# cutmix
cm_prob = 0.5
cm_beta = 1.0

# augmentation
blur_prob = 0.5
blog_sig = 0.5
jpg_prob = 0.5

best_acc = 0

# sp
sp_alpha = 0.1
sp_beta = 0.1
fc_name = 'fc.'

In [6]:
state = {}
state['num_classes'] = num_classes
state['epochs'] = epochs
state['start_epoch'] = start_epoch
state['train_batch'] = train_batch
state['test_batch'] = test_batch
state['lr'] = lr
state['schedule'] = schedule
state['momentum'] = momentum
state['gamma'] = gamma

In [7]:
def data_augment(img):
    img = np.array(img)

    if random.random() < blur_prob:
        sig = np.random.uniform(0.0, 3.0)
        gaussian_blur(img, sig)

    if random.random() < jpg_prob:
        qual = np.random.uniform(30.0, 100.0)
        img = cv2_jpg(img, qual)

    return Image.fromarray(img)


def gaussian_blur(img, sigma):
    gaussian_filter(img[:,:,0], output=img[:,:,0], sigma=sigma)
    gaussian_filter(img[:,:,1], output=img[:,:,1], sigma=sigma)
    gaussian_filter(img[:,:,2], output=img[:,:,2], sigma=sigma)


def cv2_jpg(img, compress_val):
    img_cv2 = img[:,:,::-1]
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), compress_val]
    result, encimg = cv2.imencode('.jpg', img_cv2, encode_param)
    decimg = cv2.imdecode(encimg, 1)
    return decimg[:,:,::-1]

In [8]:
def rand_bbox(size, lam):
    W = size[2]
    H = size[3]
    cut_rat = np.sqrt(1. - lam)
    cut_w = np.int(W * cut_rat)
    cut_h = np.int(H * cut_rat)

    # uniform
    cx = np.random.randint(W)
    cy = np.random.randint(H)

    bbx1 = np.clip(cx - cut_w // 2, 0, W)
    bby1 = np.clip(cy - cut_h // 2, 0, H)
    bbx2 = np.clip(cx + cut_w // 2, 0, W)
    bby2 = np.clip(cy + cut_h // 2, 0, H)

    return bbx1, bby1, bbx2, bby2

# Dataset

In [9]:
train_dir = os.path.join(source_dir, 'pggan/2000_shot_only')
val_target_dir = os.path.join(target_dir, 'validation')
val_source_dir = os.path.join(source_dir, 'validation')

train_aug = transforms.Compose([
    transforms.Lambda(lambda img: data_augment(img)),
    transforms.Resize(size),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
val_aug = transforms.Compose([
    transforms.Resize(size),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# pin_memory : cuda pin memeory use
train_loader = DataLoader(datasets.ImageFolder(train_dir, transform=train_aug),
                          batch_size=train_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_target_loader = DataLoader(datasets.ImageFolder(val_target_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)
val_source_loader = DataLoader(datasets.ImageFolder(val_source_dir, val_aug),
                       batch_size=test_batch, shuffle=True, num_workers=num_workers, pin_memory=True)

# Model

In [10]:
teacher_model = resnext50_32x4d(pretrained=False, num_classes=2)
student_model = resnext50_32x4d(pretrained=False, num_classes=2)

# Pre-trained
if pretrained:
    print("=> using pre-trained model '{}'".format(pretrained))
    teacher_model.load_state_dict(torch.load(pretrained)['state_dict'])
    student_model.load_state_dict(torch.load(pretrained)['state_dict'])

=> using pre-trained model './log/style2/128/32x4d/aug/checkpoint.pth.tar'


In [11]:
teacher_model.to('cuda')
student_model.to('cuda')
cudnn.benchmark = True
print('    Total params: %.2fM' % (sum(p.numel() for p in teacher_model.parameters())/1000000.0))

    Total params: 22.98M


In [12]:
criterion = nn.CrossEntropyLoss().cuda()
# optimizer = optim.SGD(student_model.parameters(), lr=lr, momentum=momentum)
optimizer = optim.Adam(student_model.parameters())
scheduler_cosine = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs)
scheduler_warmup = GradualWarmupScheduler(optimizer, multiplier=4, total_epoch=10, after_scheduler=scheduler_cosine)

# Loss

In [13]:
# Resume
if resume:
    print('==> Resuming from checkpoint..')
    checkpoint = os.path.dirname(resume)
#     checkpoint = torch.load(resume)
    resume = torch.load(resume)
    best_acc = resume['best_acc']
    start_epoch = resume['epoch']
    student_model.load_state_dict(resume['state_dict'])
    optimizer.load_state_dict(resume['optimizer'])
    logger = Logger(os.path.join(checkpoint, 'log.txt'), resume=True)
else:
    logger = Logger(os.path.join(checkpoint, 'log.txt'))
    logger.set_names(['Learning Rate', 'Train Loss', 'Valid Loss', 'Source Loss', 'Train Acc.', 'Valid Acc.', 'Source ACC.', 'Train AUROC', 'Valid AUROC', 'Source AUROC'])

In [14]:
for param in teacher_model.parameters():
    param.requires_grad = False
teacher_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): GroupNorm(8, 64, eps=1e-05, affine=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): GroupNorm(8, 128, eps=1e-05, affine=True)
      (conv2): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (bn2): GroupNorm(8, 128, eps=1e-05, affine=True)
      (conv3): Conv2d(128, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): GroupNorm(8, 256, eps=1e-05, affine=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (1): GroupNorm(8, 256, eps=1e-05, affine=True)
      )
    )
    (1): Bottleneck(
      (conv1): Conv2d(256, 128, k

In [15]:
teacher_model_weights = {}
for name, param in teacher_model.named_parameters():
    teacher_model_weights[name] = param.detach()

In [16]:
def reg_cls(model):
    l2_cls = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if name.startswith(fc_name):
            l2_cls += 0.5 * torch.norm(param) ** 2
    return l2_cls

def reg_l2sp(model):
    sp_loss = torch.tensor(0.).cuda()
    for name, param in model.named_parameters():
        if not name.startswith(fc_name):
            sp_loss += 0.5 * torch.norm(param - teacher_model_weights[name]) ** 2
    return sp_loss

# Train

In [17]:
def train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda):
    teacher_model.eval()
    student_model.train()
    
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    end = time.time()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()
    alpha = AverageMeter()
    
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        batch_size = inputs.size(0)
        if batch_size < train_batch:
            continue
        # measure data loading time
        data_time.update(time.time() - end)

        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
            
        r = np.random.rand(1)
        if cm_beta > 0 and r < cm_prob:
            
            rand_index = torch.randperm(inputs.size()[0]).cuda()
            tt= targets[rand_index]
            boolean = targets==tt
            rand_index = rand_index[boolean]
            lam = np.random.beta(cm_beta, cm_beta)
            bbx1, bby1, bbx2, bby2 = rand_bbox(inputs.size(), lam)
            inputs[boolean, :, bbx1:bbx2, bby1:bby2] = inputs[rand_index, :, bbx1:bbx2, bby1:bby2]

        # compute output
        with torch.no_grad():
            teacher_outputs = teacher_model(inputs)
            teacher_loss = criterion(teacher_outputs, targets)
            sp_alpha = 0
            sigmoid = nn.Sigmoid()
            sp_alpha += sigmoid(-teacher_loss)
        
        outputs = student_model(inputs)
        loss_main = criterion(outputs, targets)
        loss_cls = 0
        loss_sp = 0
        loss_cls = reg_cls(student_model)
        loss_sp = reg_l2sp(student_model)
        loss =  loss_main + sp_alpha*loss_sp + sp_alpha*loss_cls

        # measure accuracy and record loss
        prec1 = accuracy(outputs.data, targets.data)
        losses.update(loss.data.tolist(), inputs.size(0))
        top1.update(prec1[0], inputs.size(0))
        cls_losses.update(loss_cls, inputs.size(0))
        sp_losses.update(loss_sp, inputs.size(0))
        main_losses.update(loss_main.tolist(), inputs.size(0))
        alpha.update(sp_alpha, inputs.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        
#         if batch_idx % 100 == 0:
#             print('{batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
#                      batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    print('Train | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | Alpha:{alp:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, alp=alpha.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [18]:
def test(val_loader, model, criterion, epoch, use_cuda):
    global best_acc

    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    arc = AverageMeter()
    cls_losses = AverageMeter()
    sp_losses = AverageMeter()
    main_losses = AverageMeter()


    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(val_loader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # compute output
            outputs = model(inputs)
            loss_main = criterion(outputs, targets)
            loss_cls = 0
            loss_sp = 0
            loss_cls = reg_cls(model)
            loss_sp = reg_l2sp(model)
            loss = loss_main + 0*loss_sp + 0*loss_cls

            # measure accuracy and record loss
            prec1 = accuracy(outputs.data, targets.data)
            auroc = roc_auc_score(targets.cpu().detach().numpy(), outputs.cpu().detach().numpy()[:,1])
            losses.update(loss.data.tolist(), inputs.size(0))
            top1.update(prec1[0], inputs.size(0))
            arc.update(auroc, inputs.size(0))
            cls_losses.update(loss_cls, inputs.size(0))
            sp_losses.update(loss_sp, inputs.size(0))
            main_losses.update(loss_main.tolist(), inputs.size(0))


            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

    print('Test | {batch}/{size} | Loss:{loss:.4f} | MainLoss:{main:.4f} | SPLoss:{sp:.4f} | CLSLoss:{cls:.4f} | top1:{tp1:.4f} | AUROC:{ac:.4f}'.format(
                     batch=batch_idx+1, size=len(train_loader), loss=losses.avg, main=main_losses.avg, sp=sp_losses.avg, cls=cls_losses.avg, tp1=top1.avg, ac=arc.avg))
    return (losses.avg, top1.avg, arc.avg)

In [19]:
def save_checkpoint(state, is_best, checkpoint='checkpoint', filename='checkpoint.pth.tar'):
    filepath = os.path.join(checkpoint, filename)
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(checkpoint, 'model_best.pth.tar'))

def adjust_learning_rate(optimizer, epoch):
    global state
    lr_set = [1, 1e-1, 1e-2, 1e-3, 1e-4, 1e-5]
    lr_list = schedule.copy()
    lr_list.append(epoch)
    lr_list.sort()
    idx = lr_list.index(epoch)
    state['lr'] *= lr_set[idx]
    for param_group in optimizer.param_groups:
        param_group['lr'] = state['lr']

In [ ]:
for epoch in range(start_epoch, epochs):
    state['lr'] = optimizer.state_dict()['param_groups'][0]['lr']
    adjust_learning_rate(optimizer, epoch)
    print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, epochs, state['lr']))
    
    train_loss, train_acc, train_auroc = train(train_loader, teacher_model, student_model, criterion, optimizer, epoch, use_cuda)
    test_loss, test_acc, test_auroc = test(val_target_loader, student_model, criterion, epoch, use_cuda)
    source_loss, source_acc, source_auroc = test(val_source_loader, student_model, criterion, epoch, use_cuda)


    logger.append([state['lr'], train_loss, test_loss,  source_loss, train_acc, test_acc,source_acc, train_auroc, test_auroc, source_auroc])
    is_best = test_auroc+source_auroc > best_acc
    best_acc = max(test_auroc+source_auroc, best_acc)
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict' : student_model.state_dict(),
        'acc': test_acc,
        'best_acc': best_acc,
        'optimizer': optimizer.state_dict(),
    }, is_best, checkpoint=checkpoint)
    scheduler_warmup.step()
    
    teacher_model.load_state_dict(student_model.state_dict())


Epoch: [1 | 1000] LR: 0.001000
Train | 64/64 | Loss:0.9469 | MainLoss:0.5407 | Alpha:0.0090 | SPLoss:40.3306 | CLSLoss:4.7565 | top1:77.1175 | AUROC:0.0000
Test | 257/64 | Loss:0.1942 | MainLoss:0.1942 | SPLoss:14.4805 | CLSLoss:4.4144 | top1:99.4455 | AUROC:0.9999
Test | 63/64 | Loss:0.6477 | MainLoss:0.6477 | SPLoss:14.4805 | CLSLoss:4.4145 | top1:50.9231 | AUROC:0.9828

Epoch: [2 | 1000] LR: 0.001300
Train | 64/64 | Loss:1.6608 | MainLoss:0.4027 | Alpha:0.4013 | SPLoss:0.2287 | CLSLoss:2.9099 | top1:81.8286 | AUROC:0.0000
Test | 257/64 | Loss:0.1986 | MainLoss:0.1986 | SPLoss:0.1897 | CLSLoss:1.8098 | top1:98.9221 | AUROC:0.9999
Test | 63/64 | Loss:0.6639 | MainLoss:0.6639 | SPLoss:0.1897 | CLSLoss:1.8098 | top1:52.2821 | AUROC:0.9638

Epoch: [3 | 1000] LR: 0.001600
Train | 64/64 | Loss:0.9678 | MainLoss:0.4016 | Alpha:0.4020 | SPLoss:0.1960 | CLSLoss:1.2110 | top1:81.8540 | AUROC:0.0000
Test | 257/64 | Loss:0.1975 | MainLoss:0.1975 | SPLoss:0.1720 | CLSLoss:0.7748 | top1:99.0717 |


Epoch: [23 | 1000] LR: 0.003999
Train | 64/64 | Loss:0.6003 | MainLoss:0.4030 | Alpha:0.4004 | SPLoss:0.4779 | CLSLoss:0.0158 | top1:81.3714 | AUROC:0.0000
Test | 257/64 | Loss:0.1898 | MainLoss:0.1898 | SPLoss:0.2916 | CLSLoss:0.0149 | top1:99.2430 | AUROC:0.9998
Test | 63/64 | Loss:0.7271 | MainLoss:0.7271 | SPLoss:0.2916 | CLSLoss:0.0149 | top1:52.3718 | AUROC:0.8903

Epoch: [24 | 1000] LR: 0.003999
Train | 64/64 | Loss:0.5769 | MainLoss:0.4001 | Alpha:0.4015 | SPLoss:0.4248 | CLSLoss:0.0153 | top1:81.6254 | AUROC:0.0000
Test | 257/64 | Loss:0.1812 | MainLoss:0.1812 | SPLoss:0.4279 | CLSLoss:0.0143 | top1:99.6012 | AUROC:0.9998
Test | 63/64 | Loss:0.7506 | MainLoss:0.7506 | SPLoss:0.4279 | CLSLoss:0.0143 | top1:50.7564 | AUROC:0.8812

Epoch: [25 | 1000] LR: 0.003998
Train | 64/64 | Loss:0.5497 | MainLoss:0.3979 | Alpha:0.4016 | SPLoss:0.3626 | CLSLoss:0.0150 | top1:81.9556 | AUROC:0.0000
Test | 257/64 | Loss:0.1561 | MainLoss:0.1561 | SPLoss:0.3616 | CLSLoss:0.0161 | top1:99.6729 |


Epoch: [45 | 1000] LR: 0.003989
Train | 64/64 | Loss:0.5945 | MainLoss:0.3957 | Alpha:0.4009 | SPLoss:0.4837 | CLSLoss:0.0124 | top1:81.9556 | AUROC:0.0000
Test | 257/64 | Loss:0.1585 | MainLoss:0.1585 | SPLoss:0.2032 | CLSLoss:0.0116 | top1:99.5421 | AUROC:1.0000
Test | 63/64 | Loss:0.7661 | MainLoss:0.7661 | SPLoss:0.2032 | CLSLoss:0.0116 | top1:51.7564 | AUROC:0.8094

Epoch: [46 | 1000] LR: 0.003989
Train | 64/64 | Loss:0.5571 | MainLoss:0.3972 | Alpha:0.4035 | SPLoss:0.3843 | CLSLoss:0.0117 | top1:81.8286 | AUROC:0.0000
Test | 257/64 | Loss:0.1692 | MainLoss:0.1692 | SPLoss:0.7645 | CLSLoss:0.0105 | top1:99.7944 | AUROC:1.0000
Test | 63/64 | Loss:0.7648 | MainLoss:0.7648 | SPLoss:0.7645 | CLSLoss:0.0105 | top1:50.3077 | AUROC:0.8320

Epoch: [47 | 1000] LR: 0.003988
Train | 64/64 | Loss:0.5327 | MainLoss:0.3968 | Alpha:0.4016 | SPLoss:0.3272 | CLSLoss:0.0123 | top1:81.8667 | AUROC:0.0000
Test | 257/64 | Loss:0.1680 | MainLoss:0.1680 | SPLoss:0.3058 | CLSLoss:0.0152 | top1:99.1215 |


Epoch: [67 | 1000] LR: 0.000397
Train | 64/64 | Loss:0.4132 | MainLoss:0.3787 | Alpha:0.4059 | SPLoss:0.0747 | CLSLoss:0.0102 | top1:82.8952 | AUROC:0.0000
Test | 257/64 | Loss:0.1373 | MainLoss:0.1373 | SPLoss:0.0922 | CLSLoss:0.0105 | top1:99.4174 | AUROC:0.9999
Test | 63/64 | Loss:0.7992 | MainLoss:0.7992 | SPLoss:0.0922 | CLSLoss:0.0105 | top1:52.0641 | AUROC:0.9081

Epoch: [68 | 1000] LR: 0.000397
Train | 64/64 | Loss:0.4099 | MainLoss:0.3824 | Alpha:0.4054 | SPLoss:0.0578 | CLSLoss:0.0100 | top1:82.6794 | AUROC:0.0000
Test | 257/64 | Loss:0.1561 | MainLoss:0.1561 | SPLoss:0.0430 | CLSLoss:0.0103 | top1:98.8847 | AUROC:0.9999
Test | 63/64 | Loss:0.7449 | MainLoss:0.7449 | SPLoss:0.0430 | CLSLoss:0.0103 | top1:53.8205 | AUROC:0.9178

Epoch: [69 | 1000] LR: 0.000397
Train | 64/64 | Loss:0.4124 | MainLoss:0.3741 | Alpha:0.4080 | SPLoss:0.0834 | CLSLoss:0.0104 | top1:83.0730 | AUROC:0.0000
Test | 257/64 | Loss:0.1385 | MainLoss:0.1385 | SPLoss:0.0884 | CLSLoss:0.0103 | top1:99.6947 |


Epoch: [89 | 1000] LR: 0.000394
Train | 64/64 | Loss:0.4104 | MainLoss:0.3686 | Alpha:0.4083 | SPLoss:0.0925 | CLSLoss:0.0098 | top1:82.9841 | AUROC:0.0000
Test | 257/64 | Loss:0.2172 | MainLoss:0.2172 | SPLoss:0.1321 | CLSLoss:0.0101 | top1:94.3770 | AUROC:0.9999
Test | 63/64 | Loss:0.5958 | MainLoss:0.5958 | SPLoss:0.1321 | CLSLoss:0.0101 | top1:63.1538 | AUROC:0.9487

Epoch: [90 | 1000] LR: 0.000394
Train | 64/64 | Loss:0.4136 | MainLoss:0.3631 | Alpha:0.4089 | SPLoss:0.1133 | CLSLoss:0.0100 | top1:83.5556 | AUROC:0.0000
Test | 257/64 | Loss:0.1388 | MainLoss:0.1388 | SPLoss:0.1364 | CLSLoss:0.0107 | top1:98.0467 | AUROC:0.9999
Test | 63/64 | Loss:0.7710 | MainLoss:0.7710 | SPLoss:0.1364 | CLSLoss:0.0107 | top1:57.0897 | AUROC:0.9133

Epoch: [91 | 1000] LR: 0.000394
Train | 64/64 | Loss:0.4344 | MainLoss:0.3684 | Alpha:0.4088 | SPLoss:0.1512 | CLSLoss:0.0098 | top1:82.9841 | AUROC:0.0000
Test | 257/64 | Loss:0.1436 | MainLoss:0.1436 | SPLoss:0.1608 | CLSLoss:0.0088 | top1:99.0343 |

...


Epoch: [397 | 1000] LR: 0.000027
Train | 64/64 | Loss:0.0294 | MainLoss:0.0186 | Alpha:0.4958 | SPLoss:0.0122 | CLSLoss:0.0094 | top1:99.4540 | AUROC:0.0000
Test | 257/64 | Loss:0.0871 | MainLoss:0.0871 | SPLoss:0.0233 | CLSLoss:0.0094 | top1:97.2212 | AUROC:0.9998
Test | 63/64 | Loss:1.3725 | MainLoss:1.3725 | SPLoss:0.0233 | CLSLoss:0.0094 | top1:65.8974 | AUROC:0.7548

Epoch: [398 | 1000] LR: 0.000027
Train | 64/64 | Loss:0.0354 | MainLoss:0.0232 | Alpha:0.4942 | SPLoss:0.0153 | CLSLoss:0.0094 | top1:99.2889 | AUROC:0.0000
Test | 257/64 | Loss:0.1025 | MainLoss:0.1025 | SPLoss:0.0213 | CLSLoss:0.0093 | top1:96.6822 | AUROC:0.9998
Test | 63/64 | Loss:1.3033 | MainLoss:1.3033 | SPLoss:0.0213 | CLSLoss:0.0093 | top1:66.8974 | AUROC:0.7623

Epoch: [399 | 1000] LR: 0.000027
Train | 64/64 | Loss:0.0301 | MainLoss:0.0193 | Alpha:0.4953 | SPLoss:0.0125 | CLSLoss:0.0093 | top1:99.4286 | AUROC:0.0000
Test | 257/64 | Loss:0.0643 | MainLoss:0.0643 | SPLoss:0.0235 | CLSLoss:0.0093 | top1:97.862


Epoch: [419 | 1000] LR: 0.000026
Train | 64/64 | Loss:0.0290 | MainLoss:0.0184 | Alpha:0.4954 | SPLoss:0.0123 | CLSLoss:0.0091 | top1:99.4286 | AUROC:0.0000
Test | 257/64 | Loss:0.1076 | MainLoss:0.1076 | SPLoss:0.0165 | CLSLoss:0.0092 | top1:96.5358 | AUROC:0.9998
Test | 63/64 | Loss:1.2780 | MainLoss:1.2780 | SPLoss:0.0165 | CLSLoss:0.0092 | top1:67.1282 | AUROC:0.7607

Epoch: [420 | 1000] LR: 0.000026
Train | 64/64 | Loss:0.0272 | MainLoss:0.0174 | Alpha:0.4958 | SPLoss:0.0108 | CLSLoss:0.0092 | top1:99.4540 | AUROC:0.0000
Test | 257/64 | Loss:0.1069 | MainLoss:0.1069 | SPLoss:0.0130 | CLSLoss:0.0092 | top1:96.6729 | AUROC:0.9998
Test | 63/64 | Loss:1.2987 | MainLoss:1.2987 | SPLoss:0.0130 | CLSLoss:0.0092 | top1:67.2821 | AUROC:0.7601

Epoch: [421 | 1000] LR: 0.000026
Train | 64/64 | Loss:0.0263 | MainLoss:0.0165 | Alpha:0.4958 | SPLoss:0.0107 | CLSLoss:0.0092 | top1:99.4159 | AUROC:0.0000
Test | 257/64 | Loss:0.1462 | MainLoss:0.1462 | SPLoss:0.0174 | CLSLoss:0.0092 | top1:95.582


Epoch: [441 | 1000] LR: 0.000024
Train | 64/64 | Loss:0.0250 | MainLoss:0.0162 | Alpha:0.4953 | SPLoss:0.0087 | CLSLoss:0.0090 | top1:99.5175 | AUROC:0.0000
Test | 257/64 | Loss:0.0924 | MainLoss:0.0924 | SPLoss:0.0156 | CLSLoss:0.0090 | top1:97.1869 | AUROC:0.9998
Test | 63/64 | Loss:1.3494 | MainLoss:1.3494 | SPLoss:0.0156 | CLSLoss:0.0090 | top1:65.9487 | AUROC:0.7606

Epoch: [442 | 1000] LR: 0.000024
Train | 64/64 | Loss:0.0291 | MainLoss:0.0180 | Alpha:0.4956 | SPLoss:0.0134 | CLSLoss:0.0089 | top1:99.4159 | AUROC:0.0000
Test | 257/64 | Loss:0.1008 | MainLoss:0.1008 | SPLoss:0.0153 | CLSLoss:0.0089 | top1:96.9439 | AUROC:0.9998
Test | 63/64 | Loss:1.3384 | MainLoss:1.3384 | SPLoss:0.0153 | CLSLoss:0.0089 | top1:65.9359 | AUROC:0.7589

Epoch: [443 | 1000] LR: 0.000024
Train | 64/64 | Loss:0.0267 | MainLoss:0.0167 | Alpha:0.4961 | SPLoss:0.0114 | CLSLoss:0.0089 | top1:99.5048 | AUROC:0.0000
Test | 257/64 | Loss:0.1055 | MainLoss:0.1055 | SPLoss:0.0144 | CLSLoss:0.0089 | top1:96.822


Epoch: [463 | 1000] LR: 0.000023
Train | 64/64 | Loss:0.0264 | MainLoss:0.0180 | Alpha:0.4957 | SPLoss:0.0083 | CLSLoss:0.0087 | top1:99.4794 | AUROC:0.0000
Test | 257/64 | Loss:0.1369 | MainLoss:0.1369 | SPLoss:0.0128 | CLSLoss:0.0086 | top1:95.6293 | AUROC:0.9998
Test | 63/64 | Loss:1.1834 | MainLoss:1.1834 | SPLoss:0.0128 | CLSLoss:0.0086 | top1:67.7821 | AUROC:0.7608

Epoch: [464 | 1000] LR: 0.000023
Train | 64/64 | Loss:0.0274 | MainLoss:0.0179 | Alpha:0.4953 | SPLoss:0.0106 | CLSLoss:0.0086 | top1:99.5175 | AUROC:0.0000
Test | 257/64 | Loss:0.0978 | MainLoss:0.0978 | SPLoss:0.0142 | CLSLoss:0.0086 | top1:96.8567 | AUROC:0.9998
Test | 63/64 | Loss:1.2750 | MainLoss:1.2750 | SPLoss:0.0142 | CLSLoss:0.0086 | top1:66.9103 | AUROC:0.7643

Epoch: [465 | 1000] LR: 0.000023
Train | 64/64 | Loss:0.0235 | MainLoss:0.0143 | Alpha:0.4965 | SPLoss:0.0099 | CLSLoss:0.0087 | top1:99.6064 | AUROC:0.0000
Test | 257/64 | Loss:0.1088 | MainLoss:0.1088 | SPLoss:0.0118 | CLSLoss:0.0087 | top1:96.582


Epoch: [485 | 1000] LR: 0.000022
Train | 64/64 | Loss:0.0212 | MainLoss:0.0135 | Alpha:0.4967 | SPLoss:0.0071 | CLSLoss:0.0085 | top1:99.5683 | AUROC:0.0000
Test | 257/64 | Loss:0.0941 | MainLoss:0.0941 | SPLoss:0.0150 | CLSLoss:0.0085 | top1:97.0685 | AUROC:0.9998
Test | 63/64 | Loss:1.3693 | MainLoss:1.3693 | SPLoss:0.0150 | CLSLoss:0.0085 | top1:65.8077 | AUROC:0.7595

Epoch: [486 | 1000] LR: 0.000022
Train | 64/64 | Loss:0.0262 | MainLoss:0.0164 | Alpha:0.4959 | SPLoss:0.0114 | CLSLoss:0.0085 | top1:99.4667 | AUROC:0.0000
Test | 257/64 | Loss:0.0665 | MainLoss:0.0665 | SPLoss:0.0156 | CLSLoss:0.0085 | top1:97.9844 | AUROC:0.9998
Test | 63/64 | Loss:1.4842 | MainLoss:1.4842 | SPLoss:0.0156 | CLSLoss:0.0085 | top1:64.3718 | AUROC:0.7481

Epoch: [487 | 1000] LR: 0.000022
Train | 64/64 | Loss:0.0278 | MainLoss:0.0174 | Alpha:0.4950 | SPLoss:0.0126 | CLSLoss:0.0085 | top1:99.3651 | AUROC:0.0000
Test | 257/64 | Loss:0.1211 | MainLoss:0.1211 | SPLoss:0.0197 | CLSLoss:0.0085 | top1:96.199